In [10]:
import os
os.environ["SPARK_HOME"] = "/Applications/spark"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_PYTHON"] = "python"

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-ml") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()

In [8]:
import os
import pandas as pd
import numpy as np

import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, 
    StructField, 
    IntegerType, 
    FloatType, 
    StringType
)
from pyspark.sql import DataFrame

In [12]:
data = spark.read.parquet("../dataset/month_partition_online_shoppers_intention", header= True, inferSchema = True)

data.show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|  ExitRates| PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|      VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+-----------+-----------+----------+----------------+-------+------+-----------+-----------------+-------+-------------+-------------------+-----+
|             1|                   39.2|            2|                 120.8|             7|                   80.5|        0.0|       0.01|        0.0|       0.0|               

#### 1. Identify null counts for each column

In [30]:
col_name = data.columns

null_counts = data.select(
    [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in col_name]
)
null_counts.show()

+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+-----+
|Administrative|Administrative_Duration|Informational|Informational_Duration|ProductRelated|ProductRelated_Duration|BounceRates|ExitRates|PageValues|SpecialDay|OperatingSystems|Browser|Region|TrafficType|VisitorType|Weekend|made_purchase|ingestion_timestamp|Month|
+--------------+-----------------------+-------------+----------------------+--------------+-----------------------+-----------+---------+----------+----------+----------------+-------+------+-----------+-----------+-------+-------------+-------------------+-----+
|             0|                      0|            0|                     0|             0|                      0|          0|        0|         0|         0|               0|      0|     0|          0| 

#### 2. Fill null numeric values with column averages.

In [42]:
numeric_col_names = []
for col, col_type in data.dtypes:
    if col_type in ('int', 'double'):
        numeric_col_names.append(col)

print(f"length of numeric cols : {len(numeric_col_names)}")

length of numeric cols : 14
